In [1]:
# !pip install faiss-gpu
# !pip install -U sentence-transformers

In [1]:
import numpy as np
import os
import pandas as pd
import urllib.request
import faiss
import time
from sentence_transformers import SentenceTransformer, util

2023-08-19 21:48:19.324699: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:

# # 병합할 CSV 파일들이 있는 디렉토리 경로
# directory_path = './data/format/'

# # 디렉토리 내의 모든 CSV 파일을 리스트로 가져옴
# csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

# # CSV 파일들을 담을 빈 DataFrame 생성
# combined_data = pd.DataFrame()

# # 각 CSV 파일을 읽어와서 combined_data에 추가
# for file in csv_files:
#     file_path = os.path.join(directory_path, file)
#     df = pd.read_csv(file_path,encoding='cp949',index_col=0)
#     combined_data = pd.concat([combined_data, df], ignore_index=True)``

# # 결과를 하나의 CSV 파일로 저장
# combined_data.to_csv('train_combined.csv',encoding='cp949', index=False)

In [3]:
train = pd.read_csv('./data/train_combined_no_coT.csv',index_col=0,encoding='cp949')
train.head()

,sentences,morpheme,bio_tag,pos,format
0,귀의 상태에 대해 문의 주셨습니다 .,XO OOO OO OO OOOOO O,B-BD O O O O O,"(귀,NNG) / (의,JKG) / (상태,NNG) / (에,JKB) / (대해,V...",Word | POS | Entity | BIO \n귀의 |귀(NNG)+의(JKG)|...
1,현재 질문자분의 귀의 상태로 보아가벼운 염증성 질환을 의심할 수 있습니다 .,OO OOOOO XO OOO OOOOO OOO OOO OOO O OOOO O,O O B-BD O O O O O O O O,"(현재,MAG) / (질문,NNG) / (자분,NNG) / (의,JKG) / (귀,...",Word | POS | Entity | BIO \n현재 |현재(MAG)| False...
2,심하지 않은 염증소견이므로가볍게 소독하시고 지켜보시다가분비물이 나오고 있고 통증이 ...,OOO OO OOOOOOOOOO OOOOO OOOOOOOOOO OOO OO XXO ...,O O O O O O O B-ST O O O O O O O B-ST O O O O,"(심하,VA) / (지,EC) / (않,VX) / (은,ETM) / (염증,NNG)...",Word | POS | Entity | BIO \n심하지 |심하(VA)+지(EC)|...
3,답변이 도움이 되시길 바랍니다 .,OOO OOO OOO OOOO O,O O O O O,"(답변,NNG) / (이,JKS) / (도움,NNG) / (이,JKS) / (되,V...",Word | POS | Entity | BIO \n답변이 |답변(NNG)+이(JKS...
4,비행기 여행이 걱정되어 질문하신 것 같습니다 .,OOO OOO OOOO OOOO O OOOO O,O O O O O O O,"(비행기,NNG) / (여행,NNG) / (이,JKS) / (걱정,NNG) / (되...",Word | POS | Entity | BIO \n비행기 |비행기(NNG)| Fal...


In [4]:
train.iloc[0]['format'].split('\n')

['Word | POS | Entity | BIO ',
 '귀의 |귀(NNG)+의(JKG)| True | B-BD ',
 '상태에 |상태(NNG)+에(JKB)| False | O ',
 '대해 |대해(VV+EC)| False | O ',
 '문의 |문(NNG)+의(JKG)| False | O ',
 '주셨습니다 |주(VV)+셨(EP+EP)+습니다(EF)| False | O ',
 '. |.(SF)| False | O ']

In [5]:
print('train 데이터 수:', len(train))
print('train 데이터 중복제거 후:', len(set(train['sentences'])))

train 데이터 수: 8758
train 데이터 중복제거 후: 1830


In [6]:
# 'sentences' 열을 기준으로 중복된 행을 제거
train = train.drop_duplicates(subset='sentences')

# 중복 제거 후의 train 데이터 확인
print(len(train))

1830


In [7]:
from sklearn.model_selection import train_test_split

# 데이터셋을 불러온다고 가정
# dataset = pd.read_csv('your_dataset.csv')

# 데이터셋을 훈련 데이터와 테스트 데이터로 나눈다
train_data, test_data = train_test_split(train, test_size=450, random_state=42)

# 나머지 데이터는 train_data로 남아있음

# train_data와 test_data를 각각 저장한다
train_data.to_csv('./data/no_coT/train_data.csv', index=False,encoding='cp949')
test_data.to_csv('./data/no_coT/test_data.csv', index=False,encoding='cp949')

In [8]:
train_sentence = list(train_data['sentences'])
train_sentence[:10]

['대부분의 경우 특별한 치료가 필요치는 않습니다 .',
 '측두엽 간질 중에서도 측두엽 부위의 결함이나 상처가 있으면 약물조절이 어려울 수도 있습니다 .',
 '기침과 가래가 있다고 해서 천식이라고 단정 지을 수는 없으니호흡기 내과에 가셔서 흉부 사진과 폐기능 검사 등의 기초적인 검사와 진료를 받으시고정확한 진단 하에 치료를 받으시기 바랍니다 .',
 '척추관 협착증으로 질문을 주셨군요 답변 가 ) 치료 불가능합니다 .',
 '시간이 가면서 회복 될 것입니다 .',
 '두통의 빈도가 한달 5회 이상 , 두통약을 그 때마다 복용해야 하거나 효과가 없다면신경과 진료가 필요합니다 .',
 '피가 섞인 분비물이 걱정되어 질문하신 것 같습니다 .',
 '정상인의 경우 8시간 이상의 금식후 측정한 공복혈당이 100mg dL미만이고 , 식후 2시간 혈당이 140 mg dL미만입니다 .',
 '문의에 답변드리겠습니다 .',
 '허리디스크 진단을 받으셨군요 허리디스크로인해 신경의 압박이 심한 경우에는 수술이 필요합니다 .']

In [9]:
model = SentenceTransformer("jhgan/ko-sroberta-multitask")
encoded_data = model.encode(train_sentence)
print('임베딩 된 벡터 수 :', len(encoded_data))

임베딩 된 벡터 수 : 1380


In [10]:
import pickle
embedding_file_path = 'encoded_train_data.pkl'
# 임베딩된 데이터를 pickle 파일로 저장
with open('./data/pickle/'+embedding_file_path, 'wb') as f:
    pickle.dump(encoded_data, f)

In [11]:
train_list_path = 'train_list_data.pkl'
with open('./data/pickle/'+train_list_path, 'wb') as f:
    pickle.dump(train_sentence, f)

In [ ]:
eval_data = pd.read_csv('./data/eval_combined.csv',encoding='cp949')
print("eval 데이터 수",len(eval_data['sentences']))
print("eval 데이터 중복 제거 후",len(set(eval_data['sentences'])))
queries = list(set(eval_data['sentences']))

In [ ]:
queries[:5]

In [ ]:
top_k = 5
all_matched = []
for query in test_data.iloc[:2]['sentences']:
 query_embedding = model.encode(query, convert_to_tensor=True)
 cos_scores = util.pytorch_cos_sim(query_embedding, encoded_data)[0]
 cos_scores = cos_scores.cpu()

 #We use np.argpartition, to only partially sort the top_k results
 top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

 print("\n\n======================\n\n")
 print("Query:", query)
 print("\nTop 5 most similar sentences in corpus:")

 for idx in top_results[0:top_k]:
  print(train_sentence[idx].strip(), "(Score: %.4f)" % (cos_scores[idx]))
  train_loc = train.loc[train['sentences'] == train_sentence[idx]].head(1)
  train_loc




In [ ]:
outputs.append(answer)